In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import warnings
import time

from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering, MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import umap.umap_ as umap
from sklearn.decomposition import PCA

import faiss

warnings.filterwarnings("ignore")

/opt/anaconda3/envs/tdk_szakdoga/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embeddings = np.load("../data/tensors/squad_train_v2_semantic_chunking_l2.npy")

In [5]:
k_best = 180
kmeans = faiss.Kmeans(d=embeddings.shape[1], k=k_best, niter=40, verbose=True, gpu=False)
kmeans.train(embeddings)

Sampling a subset of 46080 / 84007 for training
Clustering 46080 points in 384D to 180 clusters, redo 1 times, 40 iterations
  Preprocessing in 0.10 s
  Iteration 39 (0.58 s, search 0.50 s): objective=34231.3 imbalance=1.118 nsplit=0       


np.float64(34231.3359375)

In [6]:
index_centroids = faiss.IndexFlatL2(embeddings.shape[1])
index_centroids.add(kmeans.centroids)
_, cluster_labels = index_centroids.search(embeddings, 1)
cluster_labels = cluster_labels.flatten()

In [7]:
df_semantic_train = pd.read_excel("../data/prepared/squad_train_v2_semantic_chunking.xlsx")
df_semantic_train['cluster'] = cluster_labels

In [9]:
df_semantic_train.to_excel("../data/labelled/squad_train_v2_semantic_chunking_clustered_faiss.xlsx", index=False)